In [ ]:
import numpy as np
import tensorflow as tf
import yfinance as yf
from scipy.special import gamma
from scipy.integrate import quad
import plotly.graph_objects as go
from scipy.stats import norm

# Define the Riemann-Liouville fractional derivative function
def fractional_derivative(f, alpha, t, S0, K, T, r, sigma0):
    def integrand(tau, t, alpha):
        return f(S0, K, T, r, sigma0) / (t - tau)**alpha
    integral, error = quad(integrand, 0, t, args=(t, alpha))
    return integral / gamma(1 - alpha)

# Fetch data
aapl_data = yf.Ticker('AAPL')
data = aapl_data.history(period='1Y')
data['LogReturns'] = np.log(data['Close'] / data['Close'].shift(1))
data['Volatility'] = data['LogReturns'].rolling(window=30).std()
data['AnnualizedVolatility'] = data['Volatility'] * np.sqrt(252)
data.dropna(inplace=True)

# Prepare inputs and outputs for PINN
S = data['Close'].values.astype(np.float32)
sigma = data['AnnualizedVolatility'].values.astype(np.float32)
num_data_points = len(S)
K = 180.0  # Strike price
T = 1.0    # Time to maturity in years
r = 0.04218  # Risk-free interest rate

# Fractional Black-Scholes Model parameters
alpha_times = [0.1 * i for i in range(1, 10)]

# Define the neural network model
class FAdam(tf.keras.optimizers.Adam):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, **kwargs):
        super(FAdam, self).__init__(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, **kwargs)

class PINN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='relu')
        self.dense2 = tf.keras.layers.Dense(50, activation='relu')
        self.dense3 = tf.keras.layers.Dense(50, activation='relu')
        self.dense4 = tf.keras.layers.Dense(50, activation='relu')
        self.dense5 = tf.keras.layers.Dense(50, activation='relu')
        self.dense6 = tf.keras.layers.Dense(50, activation='relu')
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        return self.out(x)

model = PINN()
optimizer = FAdam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model (we're using zero target values as a placeholder here)
history = model.fit(tf.stack([S, sigma], axis=1), np.zeros((num_data_points, 1)), epochs=2000, verbose=0)

# Predict option prices using the model
predicted_prices = model.predict(tf.stack([S, sigma], axis=1))

# Function to calculate Black-Scholes option price
def black_scholes_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return option_price

# Initialize arrays for option prices and fractional derivatives
option_prices = np.zeros((num_data_points,))
fractional_time_derivatives = {alpha: np.zeros((num_data_points,)) for alpha in alpha_times}

# Calculate fractional derivatives
for i in range(num_data_points):
    # Calculate the option price using the Black-Scholes model
    S0 = S[i]
    sigma0 = sigma[i]
    option_price = black_scholes_price(S0, K, T, r, sigma0)
    option_prices[i] = option_price

    # Calculate fractional derivatives for each alpha
    for alpha in alpha_times:
        fractional_time_derivatives[alpha][i] = fractional_derivative(black_scholes_price, alpha, T, S0, K, T, r, sigma0)

# Plotting the results using Plotly
fig = go.Figure()
for alpha in alpha_times:
    fig.add_trace(go.Scatter(x=S, y=fractional_time_derivatives[alpha], mode='markers', name=f'Option Prices for alpha = {alpha}'))
fig.update_layout(title='Predicted Option Prices for T = 1.0, K = 180, alpha = 0.1 - 0.9', xaxis_title='Stock Price', yaxis_title='Option Price')
fig.show()

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig_loss.update_layout(title='Training Loss for T = 1.0, K = 180, alpha = 0.1 - 0.9', xaxis_title='Epoch', yaxis_title='Loss')
fig_loss.show()

# Display the loss history
print("Training Loss History:", history.history['loss'])


7/7 [==============================] - 0s 3ms/step


Training Loss History: [2.7541446685791016, 0.5371072292327881, 0.129999577999115, 0.04971885308623314, 0.027158360928297043, 0.01472772378474474, 0.005616231821477413, 0.0024845441803336143, 0.001401565852575004, 0.000779572525061667, 0.00037845151382498443, 0.00017738828319124877, 8.481177064822987e-05, 3.7957783206366e-05, 2.0003521058242768e-05, 1.405728380632354e-05, 9.260146725864615e-06, 8.459340278932359e-06, 3.84013128496008e-06, 3.486769855953753e-06, 2.9397886009974172e-06, 2.927935838670237e-06, 3.0913840873836307e-06, 2.9318830456759315e-06, 4.363042080512969e-06, 4.442806584847858e-06, 4.591048309521284e-06, 3.874635694955941e-06, 3.6788137549592648e-06, 5.07907452629297e-06, 3.7249687920848373e-06, 2.9536774945881916e-06, 3.476878873698297e-06, 3.3492635793663794e-06, 3.1600143302057404e-06, 3.2070120141725056e-06, 3.2989198643917916e-06, 2.683877710296656e-06, 4.475875357456971e-06, 5.46763885722612e-06, 3.3807161798904417e-06, 2.919452981586801e-06, 3.7711265576945152e

In [ ]:
import numpy as np
import tensorflow as tf
import yfinance as yf
from scipy.special import gamma
from scipy.integrate import quad
import plotly.graph_objects as go
from scipy.stats import norm

# Define the Riemann-Liouville fractional derivative function
def fractional_derivative(f, alpha, t, S0, K, T, r, sigma0):
    def integrand(tau, t, alpha):
        return f(S0, K, T, r, sigma0) / (t - tau)**alpha
    integral, error = quad(integrand, 0, t, args=(t, alpha))
    return integral / gamma(1 - alpha)

# Fetch data
aapl_data = yf.Ticker('AAPL')
data = aapl_data.history(period='1Y')
data['LogReturns'] = np.log(data['Close'] / data['Close'].shift(1))
data['Volatility'] = data['LogReturns'].rolling(window=30).std()
data['AnnualizedVolatility'] = data['Volatility'] * np.sqrt(252)
data.dropna(inplace=True)

# Prepare inputs and outputs for PINN
S = data['Close'].values.astype(np.float32)
sigma = data['AnnualizedVolatility'].values.astype(np.float32)
num_data_points = len(S)
K = 180.0  # Strike price
T = 1.0    # Time to maturity in years
r = 0.04218  # Risk-free interest rate

# Fractional Black-Scholes Model parameters
alpha_times = [0.1 * i for i in range(1, 10)]

# Define the neural network model
class FAdam(tf.keras.optimizers.Adam):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, **kwargs):
        super(FAdam, self).__init__(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, **kwargs)

class PINN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='relu')
        self.dense2 = tf.keras.layers.Dense(50, activation='relu')
        self.dense3 = tf.keras.layers.Dense(50, activation='relu')
        self.dense4 = tf.keras.layers.Dense(50, activation='relu')
        self.dense5 = tf.keras.layers.Dense(50, activation='relu')
        self.dense6 = tf.keras.layers.Dense(50, activation='relu')
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        return self.out(x)

model = PINN()
optimizer = FAdam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model (we're using zero target values as a placeholder here)
history = model.fit(tf.stack([S, sigma], axis=1), np.zeros((num_data_points, 1)), epochs=2000, verbose=0)

# Predict option prices using the model
predicted_prices = model.predict(tf.stack([S, sigma], axis=1))

# Function to calculate Black-Scholes option price
def black_scholes_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return option_price

# Initialize arrays for option prices and fractional derivatives
option_prices = np.zeros((num_data_points,))
fractional_time_derivatives = {alpha: np.zeros((num_data_points,)) for alpha in alpha_times}

# Calculate fractional derivatives
for i in range(num_data_points):
    # Calculate the option price using the Black-Scholes model
    S0 = S[i]
    sigma0 = sigma[i]
    option_price = black_scholes_price(S0, K, T, r, sigma0)
    option_prices[i] = option_price

    # Calculate fractional derivatives for each alpha
    for alpha in alpha_times:
        fractional_time_derivatives[alpha][i] = fractional_derivative(black_scholes_price, alpha, T, S0, K, T, r, sigma0)

# Plotting the results using Plotly
fig = go.Figure()
for alpha in alpha_times:
    fig.add_trace(go.Scatter(x=S, y=fractional_time_derivatives[alpha], mode='markers', name=f'Option Prices for alpha = {alpha}'))
fig.update_layout(title='Predicted Option Prices for T = 1.0, K = 180, alpha = 0.1 - 0.9', xaxis_title='Stock Price', yaxis_title='Option Price')
fig.show()

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig_loss.update_layout(title='Training Loss for T = 1.0, K = 180, alpha = 0.1 - 0.9', xaxis_title='Epoch', yaxis_title='Loss')
fig_loss.show()

# Display the loss history
print("Training Loss History:", history.history['loss'])

# Create a surface plot of option price vs S vs T for different alpha values
S_values = np.linspace(min(S), max(S), 50)
T_values = np.linspace(0.01, 1.0, 50)  # Time to maturity from 0.01 to 1 year
S_grid, T_grid = np.meshgrid(S_values, T_values)

for alpha in alpha_times:
    option_prices_grid = np.zeros_like(S_grid)

    # Calculate fractional derivative-adjusted option prices over the grid
    for i in range(S_grid.shape[0]):
        for j in range(S_grid.shape[1]):
            option_price = black_scholes_price(S_grid[i, j], K, T_grid[i, j], r, sigma.mean())
            fractional_derivative_value = fractional_derivative(black_scholes_price, alpha, T_grid[i, j], S_grid[i, j], K, T_grid[i, j], r, sigma.mean())
            option_prices_grid[i, j] = option_price + fractional_derivative_value

    # Plot the surface
    fig_surface = go.Figure(data=[go.Surface(z=option_prices_grid, x=S_grid, y=T_grid)])
    fig_surface.update_layout(title=f'Option Price Surface for alpha = {alpha}', scene=dict(
        xaxis_title='Stock Price',
        yaxis_title='Time to Maturity',
        zaxis_title='Option Price'))
    fig_surface.show()


7/7 [==============================] - 0s 2ms/step


Training Loss History: [7.819730281829834, 0.4520522654056549, 0.2823326289653778, 0.12905529141426086, 0.06502857059240341, 0.029856376349925995, 0.012871786952018738, 0.005439674481749535, 0.0033219160977751017, 0.002165605081245303, 0.0008501945994794369, 0.00037381541915237904, 0.0002804822870530188, 0.00011069793254137039, 7.270767673617229e-05, 3.395203748368658e-05, 2.4056278562056832e-05, 1.982182584470138e-05, 1.4268795894167852e-05, 1.0884610674111173e-05, 8.407013410760555e-06, 7.661037670914084e-06, 7.179055501183029e-06, 7.469822321581887e-06, 7.852903763705399e-06, 7.861802259867545e-06, 7.419046141876606e-06, 7.574281426059315e-06, 7.427197033393895e-06, 7.516427558584837e-06, 7.211183856270509e-06, 7.902621291577816e-06, 8.374211574846413e-06, 7.814823220542166e-06, 7.256515345943626e-06, 7.330776497838087e-06, 8.489717401971575e-06, 7.580854799016379e-06, 7.987541721377056e-06, 7.022898898867425e-06, 7.3284768404846545e-06, 7.288226697710343e-06, 7.11493203198188e-06, 

In [ ]:
import numpy as np
import tensorflow as tf
import yfinance as yf
from scipy.special import gamma
from scipy.integrate import quad
import plotly.graph_objects as go
from scipy.stats import norm

# Define the Riemann-Liouville fractional derivative function
def fractional_derivative(f, alpha, t, S0, K, T, r, sigma0):
    def integrand(tau, t, alpha):
        return f(S0, K, T, r, sigma0) / (t - tau)**alpha
    integral, error = quad(integrand, 0, t, args=(t, alpha))
    return integral / gamma(1 - alpha)

# Fetch data
aapl_data = yf.Ticker('AAPL')
data = aapl_data.history(period='1Y')
data['LogReturns'] = np.log(data['Close'] / data['Close'].shift(1))
data['Volatility'] = data['LogReturns'].rolling(window=30).std()
data['AnnualizedVolatility'] = data['Volatility'] * np.sqrt(252)
data.dropna(inplace=True)

# Prepare inputs and outputs for PINN
S = data['Close'].values.astype(np.float32)
sigma = data['AnnualizedVolatility'].values.astype(np.float32)
num_data_points = len(S)
K = 180.0  # Strike price
T = 1.0    # Time to maturity in years
r = 0.04218  # Risk-free interest rate

# Fractional Black-Scholes Model parameters
alpha_times = [0.1 * i for i in range(1, 10)]

# Define the neural network model
class FAdam(tf.keras.optimizers.Adam):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, **kwargs):
        super(FAdam, self).__init__(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, **kwargs)

class PINN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='relu')
        self.dense2 = tf.keras.layers.Dense(50, activation='relu')
        self.dense3 = tf.keras.layers.Dense(50, activation='relu')
        self.dense4 = tf.keras.layers.Dense(50, activation='relu')
        self.dense5 = tf.keras.layers.Dense(50, activation='relu')
        self.dense6 = tf.keras.layers.Dense(50, activation='relu')
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        return self.out(x)

model = PINN()
optimizer = FAdam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model (we're using zero target values as a placeholder here)
history = model.fit(tf.stack([S, sigma], axis=1), np.zeros((num_data_points, 1)), epochs=2000, verbose=0)

# Predict option prices using the model
predicted_prices = model.predict(tf.stack([S, sigma], axis=1))

# Function to calculate Black-Scholes option price
def black_scholes_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return option_price

# Initialize arrays for option prices and fractional derivatives
option_prices = np.zeros((num_data_points,))
fractional_time_derivatives = {alpha: np.zeros((num_data_points,)) for alpha in alpha_times}

# Calculate fractional derivatives
for i in range(num_data_points):
    # Calculate the option price using the Black-Scholes model
    S0 = S[i]
    sigma0 = sigma[i]
    option_price = black_scholes_price(S0, K, T, r, sigma0)
    option_prices[i] = option_price

    # Calculate fractional derivatives for each alpha
    for alpha in alpha_times:
        fractional_time_derivatives[alpha][i] = fractional_derivative(black_scholes_price, alpha, T, S0, K, T, r, sigma0)

# Plotting the results using Plotly
fig = go.Figure()
for alpha in alpha_times:
    fig.add_trace(go.Scatter(x=S, y=fractional_time_derivatives[alpha], mode='markers', name=f'Option Prices for alpha = {alpha}'))
fig.update_layout(title='Predicted Option Prices for T = 1.0, K = 180, alpha = 0.1 - 0.9', xaxis_title='Stock Price', yaxis_title='Option Price')
fig.show()

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig_loss.update_layout(title='Training Loss for T = 1.0, K = 180, alpha = 0.1 - 0.9', xaxis_title='Epoch', yaxis_title='Loss')
fig_loss.show()

# Display the loss history
print("Training Loss History:", history.history['loss'])

# Create a surface plot of option price vs S vs T for different alpha values
S_values = np.linspace(min(S), max(S), 50)
T_values = np.linspace(0.01, 1.0, 50)  # Time to maturity from 0.01 to 1 year
S_grid, T_grid = np.meshgrid(S_values, T_values)

fig_surface = go.Figure()

for alpha in alpha_times:
    option_prices_grid = np.zeros_like(S_grid)

    # Calculate fractional derivative-adjusted option prices over the grid
    for i in range(S_grid.shape[0]):
        for j in range(S_grid.shape[1]):
            option_price = black_scholes_price(S_grid[i, j], K, T_grid[i, j], r, sigma.mean())
            fractional_derivative_value = fractional_derivative(black_scholes_price, alpha, T_grid[i, j], S_grid[i, j], K, T_grid[i, j], r, sigma.mean())
            option_prices_grid[i, j] = option_price + fractional_derivative_value

    # Add the surface to the figure
    fig_surface.add_trace(go.Surface(z=option_prices_grid, x=S_grid, y=T_grid, showscale=False, name=f'alpha = {alpha}'))

fig_surface.update_layout(title='Option Price Surfaces for Different Alpha Values', scene=dict(
    xaxis_title='Stock Price',
    yaxis_title='Time to Maturity',
    zaxis_title='Option Price'))
fig_surface.show()

7/7 [==============================] - 0s 3ms/step


Training Loss History: [7.740344524383545, 0.6009077429771423, 0.28068625926971436, 0.13523876667022705, 0.05838983878493309, 0.02274048514664173, 0.004452233202755451, 0.0022505847737193108, 0.001132158562541008, 0.000542040856089443, 0.000266673305304721, 0.00012808360042981803, 6.716900679748505e-05, 3.1808951462153345e-05, 1.534586408524774e-05, 8.707333108759485e-06, 3.881556949636433e-06, 3.1953679808793822e-06, 3.470015144557692e-06, 3.1692011361883488e-06, 2.383908849878935e-06, 2.115360075549688e-06, 2.151965418306645e-06, 2.0842539925070014e-06, 2.0865707028860925e-06, 2.1486907826329116e-06, 2.0848212898272322e-06, 2.2385293050319888e-06, 2.1056216610304546e-06, 2.1124942577444017e-06, 2.073394398394157e-06, 2.113671143888496e-06, 2.0568825220834697e-06, 2.0981613033654867e-06, 2.0772224615939194e-06, 2.1250250483717537e-06, 2.4642847620270913e-06, 2.4484390905854525e-06, 2.3392988168779993e-06, 2.245927362309885e-06, 2.1072471554361982e-06, 2.1040868887212127e-06, 2.3734053

In [7]:
import numpy as np
import tensorflow as tf
import yfinance as yf
from scipy.special import gamma
from scipy.integrate import quad
import plotly.graph_objects as go
from scipy.stats import norm

# Define the Riemann-Liouville fractional derivative function
# def fractional_derivative(f, alpha, t, S0, K, T, r, sigma0):
#     def integrand(tau, t, alpha):
#         return f(S0, K, T, r, sigma0) / (t - tau)**alpha
#     integral, error = quad(integrand, 0, t, args=(t, alpha))
#     return integral / gamma(1 - alpha)

# Fetch data
aapl_data = yf.Ticker('AAPL')
data = aapl_data.history(period='1Y')
data['LogReturns'] = np.log(data['Close'] / data['Close'].shift(1))
data['Volatility'] = data['LogReturns'].rolling(window=30).std()
data['AnnualizedVolatility'] = data['Volatility'] * np.sqrt(252)
data.dropna(inplace=True)

# Prepare inputs and outputs for PINN
S = data['Close'].values.astype(np.float32)
sigma = data['AnnualizedVolatility'].values.astype(np.float32)
num_data_points = len(S)
K = 180.0  # Strike price
T = 1.0    # Time to maturity in years
r = 0.04218  # Risk-free interest rate

# Fractional Black-Scholes Model parameters
alpha_times = [0.1 * i for i in range(10, 11)]

# Define the neural network model
class FAdam(tf.keras.optimizers.Adam):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, **kwargs):
        super(FAdam, self).__init__(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, **kwargs)

class PINN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='relu')
        self.dense2 = tf.keras.layers.Dense(50, activation='relu')
        self.dense3 = tf.keras.layers.Dense(50, activation='relu')
        self.dense4 = tf.keras.layers.Dense(50, activation='relu')
        self.dense5 = tf.keras.layers.Dense(50, activation='relu')
        self.dense6 = tf.keras.layers.Dense(50, activation='relu')
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        return self.out(x)

model = PINN()
optimizer = FAdam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model (we're using zero target values as a placeholder here)
history = model.fit(tf.stack([S, sigma], axis=1), np.zeros((num_data_points, 1)), epochs=2000, verbose=0)

# Predict option prices using the model
predicted_prices = model.predict(tf.stack([S, sigma], axis=1))

# Function to calculate Black-Scholes option price
def black_scholes_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return option_price

# # Initialize arrays for option prices and fractional derivatives
# option_prices = np.zeros((num_data_points,))
# fractional_time_derivatives = {alpha: np.zeros((num_data_points,)) for alpha in alpha_times}

# Calculate fractional derivatives
for i in range(num_data_points):
    # Calculate the option price using the Black-Scholes model
    S0 = S[i]
    sigma0 = sigma[i]
    option_price = black_scholes_price(S0, K, T, r, sigma0)
    option_prices[i] = option_price

    # # Calculate fractional derivatives for each alpha
    # for alpha in alpha_times:
    #     fractional_time_derivatives[alpha][i] = fractional_derivative(black_scholes_price, alpha, T, S0, K, T, r, sigma0)

# Plotting the results using Plotly
fig = go.Figure()
for alpha in alpha_times:
    fig.add_trace(go.Scatter(x=S, y=option_prices, mode='markers', name=f'Option Prices for alpha = {alpha}'))
fig.update_layout(title='Predicted Option Prices for T = 1.0, K = 180, alpha = 1.0', xaxis_title='Stock Price', yaxis_title='Option Price')
fig.show()

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig_loss.update_layout(title='Training Loss for T = 1.0, K = 180, alpha = 1.0', xaxis_title='Epoch', yaxis_title='Loss')
fig_loss.show()

# Display the loss history
print("Training Loss History:", history.history['loss'])

# # Create a surface plot of option price vs S vs T for different alpha values
# S_values = np.linspace(min(S), max(S), 50)
# T_values = np.linspace(0.01, 1.0, 50)  # Time to maturity from 0.01 to 1 year
# S_grid, T_grid = np.meshgrid(S_values, T_values)

# colorscales = ['Viridis', 'Cividis', 'Plasma', 'Inferno', 'Magma', 'Greys', 'YlGnBu', 'YlOrRd', 'Turbo']

# fig_surface = go.Figure()

# for i, alpha in enumerate(alpha_times):
#     option_prices_grid = np.zeros_like(S_grid)

#     # Calculate fractional derivative-adjusted option prices over the grid
#     for i in range(S_grid.shape[0]):
#         for j in range(S_grid.shape[1]):
#             option_price = black_scholes_price(S_grid[i, j], K, T_grid[i, j], r, sigma.mean())
#             fractional_derivative_value = fractional_derivative(black_scholes_price, alpha, T_grid[i, j], S_grid[i, j], K, T_grid[i, j], r, sigma.mean())
#             option_prices_grid[i, j] = option_price + fractional_derivative_value

#     # Add the surface to the figure
#     fig_surface.add_trace(go.Surface(z=option_prices_grid, x=S_grid, y=T_grid, colorscale=colorscales[i % len(colorscales)], showscale=False, name=f'alpha = {alpha}'))

# fig_surface.update_layout(title='Option Price Surfaces for Different Alpha Values', scene=dict(
#     xaxis_title='Stock Price',
#     yaxis_title='Time to Maturity',
#     zaxis_title='Option Price'))
# fig_surface.show()


7/7 [==============================] - 0s 2ms/step


Training Loss History: [4.308929443359375, 0.4766905903816223, 0.18569958209991455, 0.1380331814289093, 0.06500649452209473, 0.019447777420282364, 0.010402374900877476, 0.007533314172178507, 0.003906724043190479, 0.001615922781638801, 0.0006250940496101975, 0.0002966706524603069, 0.00015269672439899296, 8.207307837437838e-05, 4.693190930993296e-05, 2.5749974156497046e-05, 1.2874479580204934e-05, 7.966713383211754e-06, 4.423648988449713e-06, 3.21381139656296e-06, 2.960884785352391e-06, 2.464774979671347e-06, 2.422266106805182e-06, 2.214547066614614e-06, 2.15226555155823e-06, 2.236649379483424e-06, 2.151836724806344e-06, 2.1652726900356356e-06, 2.1530167941818945e-06, 2.1588950858131284e-06, 2.122325895470567e-06, 2.245880523332744e-06, 2.321504553037812e-06, 2.2495353277918184e-06, 2.1000762444600696e-06, 2.387357653788058e-06, 2.269806373078609e-06, 2.3076884190231794e-06, 2.227072400273755e-06, 2.1878738607483683e-06, 2.195119122916367e-06, 2.1289206415531226e-06, 2.2722254016116494e-

In [ ]:
# Define a list of distinct colors
colors = [
    'rgb(255, 0, 0)',      # Red
    'rgb(0, 255, 0)',      # Green
    'rgb(0, 0, 255)',      # Blue
    'rgb(255, 255, 0)',    # Yellow
    'rgb(255, 165, 0)',    # Orange
    'rgb(128, 0, 128)',    # Purple
    'rgb(0, 255, 255)',    # Cyan
    'rgb(255, 20, 147)',   # Deep Pink
    'rgb(139, 69, 19)'     # Brown
]
fig_surface = go.Figure()

for i, alpha in enumerate(alpha_times):
    option_prices_grid = np.zeros_like(S_grid)

    # Calculate fractional derivative-adjusted option prices over the grid
    for row in range(S_grid.shape[0]):
        for col in range(S_grid.shape[1]):
            option_price = black_scholes_price(S_grid[row, col], K, T_grid[row, col], r, sigma.mean())
            fractional_derivative_value = fractional_derivative(black_scholes_price, alpha, T_grid[row, col], S_grid[row, col], K, T_grid[row, col], r, sigma.mean())
            option_prices_grid[row, col] = option_price + fractional_derivative_value

    # Add the surface to the figure
    fig_surface.add_trace(go.Surface(z=option_prices_grid, x=S_grid, y=T_grid, surfacecolor=option_prices_grid, colorscale=[[0, colors[i]], [1, colors[i]]], showscale=False, name=f'alpha = {alpha}'))

fig_surface.update_layout(title='Option Price Surfaces for Different Alpha Values', scene=dict(
    xaxis_title='Stock Price',
    yaxis_title='Time to Maturity',
    zaxis_title='Option Price'))
fig_surface.show()

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf
import yfinance as yf
from scipy.special import gamma
from scipy.integrate import quad
import plotly.graph_objects as go
from scipy.stats import norm

# Define the Riemann-Liouville fractional derivative function
def fractional_derivative(f, alpha, t, S0, K, T, r, sigma0):
    def integrand(tau, t, alpha):
        return f(S0, K, T, r, sigma0) / (t - tau)**alpha

    integral, error = quad(integrand, 0, t, args=(t, alpha))
    return integral / gamma(1 - alpha)

# Fetch data
aapl_data = yf.Ticker('AAPL')
data = aapl_data.history(period='9mo')
data['LogReturns'] = np.log(data['Close'] / data['Close'].shift(1))
data['Volatility'] = data['LogReturns'].rolling(window=30).std()
data['AnnualizedVolatility'] = data['Volatility'] * np.sqrt(252)
data.dropna(inplace=True)

# Prepare inputs and outputs for PINN
S = data['Close'].values.astype(np.float32)
sigma = data['AnnualizedVolatility'].values.astype(np.float32)
num_data_points = len(S)
K = 130.0  # Strike price
T = 1.0    # Time to maturity in years
r = 0.04218  # Risk-free interest rate

# Fractional Black-Scholes Model parameters
alpha_times = [1.0 * i for i in range(1, 2)]

# Define the neural network model
class FAdam(tf.keras.optimizers.Adam):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, **kwargs):
        super(FAdam, self).__init__(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, **kwargs)

class PINN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='relu')
        self.dense2 = tf.keras.layers.Dense(50, activation='relu')
        self.dense3 = tf.keras.layers.Dense(50, activation='relu')
        self.dense4 = tf.keras.layers.Dense(50, activation='relu')
        self.dense5 = tf.keras.layers.Dense(50, activation='relu')
        self.dense6 = tf.keras.layers.Dense(50, activation='relu')
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        return self.out(x)

model = PINN()
optimizer = FAdam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model (we're using zero target values as a placeholder here)
history = model.fit(tf.stack([S, sigma], axis=1), np.zeros((num_data_points, 1)), epochs=2000, verbose=0)

# Predict option prices using the model
predicted_prices = model.predict(tf.stack([S, sigma], axis=1))

# Function to calculate Black-Scholes option price
def black_scholes_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return option_price

# Initialize arrays for option prices and fractional derivatives
option_prices = np.zeros((num_data_points,))
fractional_time_derivatives = {alpha: np.zeros((num_data_points,)) for alpha in alpha_times}

# Calculate fractional derivatives
for i in range(num_data_points):
    # Calculate the option price using the Black-Scholes model
    S0 = S[i]
    sigma0 = sigma[i]
    option_price = black_scholes_price(S0, K, T, r, sigma0)
    option_prices[i] = option_price

    # Calculate fractional derivatives for each alpha
    for alpha in alpha_times:
        fractional_time_derivatives[alpha][i] = fractional_derivative(black_scholes_price, alpha, T, S0, K, T, r, sigma0)

# Plotting the results using Plotly
fig = go.Figure()
for alpha in alpha_times:
    fig.add_trace(go.Scatter(x=S, y=fractional_time_derivatives[alpha], mode='markers', name=f'Option Price for alpha = {alpha}'))

fig.update_layout(
    title='Option Prices for T = 1.0, K = 130, alpha = 0.1 - 0.9',
    xaxis_title='Stock Price',
    yaxis_title='Option Price',
    template='plotly_white'
)
fig.show()

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig_loss.update_layout(
    title='Training Loss for T = 1.0, K = 130, alpha = 0.1 - 0.9',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    template='plotly_white'
)
fig_loss.show()

# Display the loss history
print("Training Loss History:", history.history['loss'])


ERROR:yfinance:AAPL: Period '9mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']


ValueError: Expected input data to be non-empty.

In [ ]:
import numpy as np
import tensorflow as tf
import yfinance as yf
from scipy.special import gamma
from scipy.integrate import quad
import plotly.graph_objects as go
from scipy.stats import norm

# Fetch data
aapl_data = yf.Ticker('AAPL')
data = aapl_data.history(period='9mo')
data['LogReturns'] = np.log(data['Close'] / data['Close'].shift(1))
data['Volatility'] = data['LogReturns'].rolling(window=30).std()
data['AnnualizedVolatility'] = data['Volatility'] * np.sqrt(252)
data.dropna(inplace=True)

# Prepare inputs and outputs for PINN
S = data['Close'].values.astype(np.float32)
sigma = data['AnnualizedVolatility'].values.astype(np.float32)
num_data_points = len(S)
K = 180.0  # Strike price
T = 2.5    # Time to maturity in years
r = 0.04218  # Risk-free interest rate

# Define the neural network model
class FAdam(tf.keras.optimizers.Adam):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, **kwargs):
        super(FAdam, self).__init__(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, **kwargs)

class PINN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='relu')
        self.dense2 = tf.keras.layers.Dense(50, activation='relu')
        self.dense3 = tf.keras.layers.Dense(50, activation='relu')
        self.dense4 = tf.keras.layers.Dense(50, activation='relu')
        self.dense5 = tf.keras.layers.Dense(50, activation='relu')
        self.dense6 = tf.keras.layers.Dense(50, activation='relu')
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        return self.out(x)

model = PINN()
optimizer = FAdam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model (we're using zero target values as a placeholder here)
history = model.fit(tf.stack([S, sigma], axis=1), np.zeros((num_data_points, 1)), epochs=2000, verbose=0)

# Predict option prices using the model
predicted_prices = model.predict(tf.stack([S, sigma], axis=1))

# Function to calculate Black-Scholes option price
def black_scholes_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return option_price

# Define the Riemann-Liouville fractional derivative function
def fractional_derivative(f, alpha, t, *args):
    def integrand(tau, t, alpha, *args):
        return f(*args) / (t - tau)**alpha

    integral, error = quad(integrand, 0, t, args=(t, alpha, *args))
    return integral / gamma(1 - alpha)

# Initialize arrays for option prices and fractional derivatives
option_prices = np.zeros((num_data_points,))
fractional_price_derivatives = np.zeros((num_data_points,))

# Calculate fractional derivatives
alpha_price = 1.0  # Order of price derivative (regular derivative)

for i in range(num_data_points):
    # Calculate the option price using the Black-Scholes model
    S0 = S[i]
    sigma0 = sigma[i]
    option_price = black_scholes_price(S0, K, T, r, sigma0)
    option_prices[i] = option_price

    # Calculate fractional derivatives (for price only)
    fractional_price_deriv = fractional_derivative(black_scholes_price, alpha_price, S0, S0, K, T, r, sigma0)
    fractional_price_derivatives[i] = fractional_price_deriv

# Plotting the results using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=S, y=option_prices, mode='markers', name='Option Prices for alpha = 1.0'))
# fig.add_trace(go.Scatter(x=S, y=predicted_prices.flatten(), mode='lines', name='Predicted Option Prices'))

fig.update_layout(title='Option Prices for T = 2.5, K = 180, alpha = 1.0', xaxis_title='Stock Price', yaxis_title='Option Price',
    template='plotly_white')
fig.show()

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig_loss.update_layout(title='Training Loss for T = 2.5, K = 180, alpha = 1.0', xaxis_title='Epoch', yaxis_title='Loss',
    template='plotly_white')

fig_loss.show()

# Display the loss history
print("Training Loss History:", history.history['loss'])
